# Getting started

First, we enable the cluster to scale up. Note that if you run an auto-scaling cluster,
Google will suspend your nodes. Make sure to have the experiment prepared before running the commands.

The following is assumed ready:
* GKE/Kubernetes cluster (see also `terraform/terraform_notebook.ipynb`)
    * 2 nodes pools (default for system & dependencies, experiment pool)
* Docker image (including dataset, to speed-up starting experiments).
    * Within a bash shell
        * Make sure to have the `requirements-cpu.txt` installed (or `requirements-gpu.txt (in a virtual venv/conda environment). You can run `pip3 install -r requirements-cpu.txt`
    * First run the extractor (locally) `python3 -m fltk extractor configs/example_cloud_experiment.json`
        *  This downloads datasets to be included in the docker image.
    * Build the container `DOCKER_BUILDKIT=1 docker build --platform linux/amd64 . --tag gcr.io/test-bed-fltk-jerrit/fltk`
    * Push to your gcr.io repository `docker push gcr.io/test-bed-fltk-jerrit/fltk`


With that setup, first set some variables used throughout the experiment.


In [1]:
##################
### CHANGE ME! ###
##################
PROJECT_ID="fltk-flint" # changed
CLUSTER_NAME="fltk-testbed-cluster"
DEFAULT_POOL="default-node-pool"
#EXPERIMENT_POOL="medium-fltk-pool-1"
EXPERIMENT_POOL="small-fltk-pool-1" # changed
REGION="us-central1-c"

# In case we do not yet have the credentials/kubeconfig
gcloud container clusters get-credentials $CLUSTER_NAME --region $REGION --project $PROJECT_ID

Fetching cluster endpoint and auth data.
kubeconfig entry generated for fltk-testbed-cluster.


Scale the default-node-pool up.

In [2]:
# These commands might take a while to complete.
gcloud container clusters resize $CLUSTER_NAME --node-pool $DEFAULT_POOL \
     --num-nodes 1 --region $REGION --quiet
     

Resizing fltk-testbed-cluster...done.


Updated [https://container.googleapis.com/v1/projects/test-bed-fltk-jerrit/zones/us-central1-c/clusters/fltk-testbed-cluster].


In [3]:
# These commands might take a while to complete.
gcloud container clusters resize $CLUSTER_NAME --node-pool $EXPERIMENT_POOL \
     --num-nodes 1 --region $REGION --quiet

Resizing fltk-testbed-cluster...done.


Updated [https://container.googleapis.com/v1/projects/test-bed-fltk-jerrit/zones/us-central1-c/clusters/fltk-testbed-cluster].


## Preparation
In case you have already tested something or ran another experiment, we have to remove the deployment of the Orchestrator. This will not delete any experiment data, as this persists on one of the ReadWriteMany PVCs.


Currently, the Orchestrator is deployed using a `Deployment` definition, a future version will replace this with a `Deployment` definition, to make this step unnecessary. For experiments this means the following:

1. A single deployment can exist at a single time in a single namespace. This includes 'completed' experiments.
2. For running batches of experiments, a BatchOrchestrator is provided.


ℹ️ This will not remove any data, but if your orchestrator is still/already running experiments, this will stop the deployment. Running training jobs will not be stopped, for this you can use `kubectl`. ConfigMaps created by the Orchestrator (to provide experiment configurations), will not be removed. See the commented code in the cell below.

In [104]:
# If you want to delete all pytorch trainjobs, uncomment the command below.
# kubectl delete pytorchjobs.kubeflow.org --all --namespace test

# If you want to delete all existing configuration map objects in a namespace, run teh command below
# kubectl delete configmaps --all --namespace test

helm uninstall -n test flearner

W1008 15:26:49.577866   11165 gcp.go:120] WARNING: the gcp auth plugin is deprecated in v1.22+, unavailable in v1.25+; use gcloud instead.
To learn more, consult https://cloud.google.com/blog/products/containers-kubernetes/kubectl-auth-changes-in-gke
Error: uninstall: Release not loaded: flearner: release: not found


: 1

## Install extractor

Deploy the TensorBoard service and persistent volumes, required for deployment of the orchestrator's chart.

In [4]:
helm upgrade --install -n test extractor ../charts/extractor -f ../charts/fltk-values.yaml \
    --set provider.projectName=$PROJECT_ID

W1010 17:48:46.808570   10922 gcp.go:120] WARNING: the gcp auth plugin is deprecated in v1.22+, unavailable in v1.25+; use gcloud instead.
To learn more, consult https://cloud.google.com/blog/products/containers-kubernetes/kubectl-auth-changes-in-gke
Release "extractor" does not exist. Installing it now.
NAME: extractor
LAST DEPLOYED: Mon Oct 10 17:48:50 2022
NAMESPACE: test
STATUS: deployed
REVISION: 1
TEST SUITE: None
NOTES:
Get the FLTK extractors Tensorboard URL by running:

export POD_NAME=$(kubectl get pods -n test -l "app.kubernetes.io/name=fltk.extractor" -o jsonpath="{.items[0].metadata.name}")
echo http://localhost:6006/
kubectl -n test port-forward $POD_NAME 6006:6006


## Define experiment configuration files

Deployment of experiments is currently done through a Helm Deployment. A future release (™️) will rework this to a Job definition, as this allows to re-use the template more easily.


> The `EXPERIMENT_FILE` will contain the description of the experiments
> The `CLUSTER_CONFIG` will contain shared configurations for logging, Orchestrator configuration and replication information.

In [5]:
EXPERIMENT_FILE="../configs/distributed_tasks/vision_transformer_experiment.json"
CLUSTER_CONFIG="../configs/vision_transformer_experiment.json"

## Setup experiment variables
Next, we will deploy the experiments.


We provide a configuration file, `charts/fltk-values.yaml`, in here change the values under the `provider` block. Change `projectName` to your Google Cloud Project ID.

```yaml
provider:
    domain: gcr.io
    projectName: CHANGE_ME!
    imageName: fltk:latest
```

We use the `--set-file` flag for `helm`, as currently, Helm does not support using files outside of the chart root directory (in this case `charts/orchestrator`). Using `--set-file` we can dynamically provide these files. See also issue [here](https://github.com/helm/helm/issues/3276)


In [9]:
helm uninstall -n test experiment-orchestrator

W1010 18:14:13.415457   13804 gcp.go:120] WARNING: the gcp auth plugin is deprecated in v1.22+, unavailable in v1.25+; use gcloud instead.
To learn more, consult https://cloud.google.com/blog/products/containers-kubernetes/kubectl-auth-changes-in-gke
release "experiment-orchestrator" uninstalled


In [10]:
helm install -n test experiment-orchestrator ../charts/orchestrator -f ../charts/fltk-values.yaml \
    --set-file orchestrator.experiment=$EXPERIMENT_FILE,orchestrator.configuration=$CLUSTER_CONFIG \
    --set provider.projectName=$PROJECT_ID

W1010 18:14:17.577794   13818 gcp.go:120] WARNING: the gcp auth plugin is deprecated in v1.22+, unavailable in v1.25+; use gcloud instead.
To learn more, consult https://cloud.google.com/blog/products/containers-kubernetes/kubectl-auth-changes-in-gke
NAME: experiment-orchestrator
LAST DEPLOYED: Mon Oct 10 18:14:20 2022
NAMESPACE: test
STATUS: deployed
REVISION: 1
TEST SUITE: None
NOTES:
You successfully launched an experiment configuration on your cluster in test.

N.B. Make sure to collect all data after completing your experiment!
N.B. Re-installing the orchestrator WILL RESULT IN DELETION OF ALL TRAINJOBS and PODS!


In [13]:
# To get logs from the orchestrator
kubectl logs -n test fl-server

10-10-2022 16:14:28 root         INFO     Loading file config/configuration.fltk.json
No argument path is provided.
10-10-2022 16:14:28 root         INFO     Starting in cluster mode.
10-10-2022 16:14:28 root         INFO     Starting with experiment replication: 0 with seed: 42
10-10-2022 16:14:28 root         INFO     Starting as Orchestrator
10-10-2022 16:14:28 root         INFO     Starting Orchestrator, initializing resources....
10-10-2022 16:14:28 root         INFO     Loading in cluster configuration file
10-10-2022 16:14:28 root         INFO     Pointing configuration to in cluster configuration.
10-10-2022 16:14:28 root         INFO     Starting cluster manager
10-10-2022 16:14:28 ClusterManager INFO     Spinning up cluster manager...
10-10-2022 16:14:28 ResourceWatchDog INFO     Starting resource watchdog
10-10-2022 16:14:28 ResourceWatchDog INFO     Fetching node information of cluster...
10-10-2022 16:14:28 root         INFO     Starting arrival generator
10-10-2022 16:14:

10-10-2022 16:17:35 ResourceWatchDog INFO     Fetching pod information of cluster...
10-10-2022 16:17:39 root         INFO     Waiting for ccc6572c-3f39-4df8-9175-106a0b8d44a7 to complete, 0 pending, 0 arrivals
10-10-2022 16:17:44 root         INFO     Waiting for ccc6572c-3f39-4df8-9175-106a0b8d44a7 to complete, 0 pending, 0 arrivals
10-10-2022 16:17:45 ResourceWatchDog INFO     Fetching pod information of cluster...
10-10-2022 16:17:49 root         INFO     Waiting for ccc6572c-3f39-4df8-9175-106a0b8d44a7 to complete, 0 pending, 0 arrivals
10-10-2022 16:17:54 root         INFO     Waiting for ccc6572c-3f39-4df8-9175-106a0b8d44a7 to complete, 0 pending, 0 arrivals
10-10-2022 16:17:55 ResourceWatchDog INFO     Fetching pod information of cluster...
10-10-2022 16:17:59 root         INFO     Waiting for ccc6572c-3f39-4df8-9175-106a0b8d44a7 to complete, 0 pending, 0 arrivals
10-10-2022 16:18:04 root         INFO     Waiting for ccc6572c-3f39-4df8-9175-106a0b8d44a7 to complete, 0 pending, 

In [20]:
# To get logs from learners (example)
kubectl logs -n test trainjob-ccc6572c-3f39-4df8-9175-106a0b8d44a7-master-0

# To get logs from learners (federated learning)
#kubectl logs -n test trainjob-f8a9e273-9267-49b3-928b-e83599caf996-worker-0 

10-10-2022 16:14:33 root         INFO     Loading file config/configuration.fltk.json
10-10-2022 16:14:33 root         INFO     Starting in client mode
10-10-2022 16:14:33 root         INFO     Starting with host=trainjob-ccc6572c-3f39-4df8-9175-106a0b8d44a7-master-0 and port=23456
10-10-2022 16:14:33 root         INFO     Starting Creating client with 0
10-10-2022 16:14:33 Client-0-ccc6572c-3f39-4df8-9175-106a0b8d44a7 INFO     Initializing learning client
10-10-2022 16:14:34 root         INFO     Getting net: Nets.vit_mnist
10-10-2022 16:14:34 Client-0-ccc6572c-3f39-4df8-9175-106a0b8d44a7 INFO     Preparing learner model with distributed=False
10-10-2022 16:14:34 Client-0-ccc6572c-3f39-4df8-9175-106a0b8d44a7 INFO     Starting training of model Nets.vit_mnist with batch size 100 and max epochs 25.
10-10-2022 16:14:36 Client-0-ccc6572c-3f39-4df8-9175-106a0b8d44a7 INFO     [1,     0] loss: 0.046
10-10-2022 16:14:46 Client-0-ccc6572c-3f39-4df8-9175-106a0b8d44a7 INFO     [1,    50] loss: 1

10-10-2022 16:27:58 Client-0-ccc6572c-3f39-4df8-9175-106a0b8d44a7 INFO     [7,   200] loss: 0.065
10-10-2022 16:28:08 Client-0-ccc6572c-3f39-4df8-9175-106a0b8d44a7 INFO     [7,   250] loss: 0.064
10-10-2022 16:28:18 Client-0-ccc6572c-3f39-4df8-9175-106a0b8d44a7 INFO     [7,   300] loss: 0.070
10-10-2022 16:28:27 Client-0-ccc6572c-3f39-4df8-9175-106a0b8d44a7 INFO     [7,   350] loss: 0.061
10-10-2022 16:28:37 Client-0-ccc6572c-3f39-4df8-9175-106a0b8d44a7 INFO     [7,   400] loss: 0.072
10-10-2022 16:28:47 Client-0-ccc6572c-3f39-4df8-9175-106a0b8d44a7 INFO     [7,   450] loss: 0.064
10-10-2022 16:28:57 Client-0-ccc6572c-3f39-4df8-9175-106a0b8d44a7 INFO     [7,   500] loss: 0.059
10-10-2022 16:29:07 Client-0-ccc6572c-3f39-4df8-9175-106a0b8d44a7 INFO     [7,   550] loss: 0.067
10-10-2022 16:29:26 Client-0-ccc6572c-3f39-4df8-9175-106a0b8d44a7 INFO     [8,     0] loss: 0.001
10-10-2022 16:29:36 Client-0-ccc6572c-3f39-4df8-9175-106a0b8d44a7 INFO     [8,    50] loss: 0.049
10-10-2022 16:29:46 

10-10-2022 16:42:50 Client-0-ccc6572c-3f39-4df8-9175-106a0b8d44a7 INFO     [14,   200] loss: 0.039
10-10-2022 16:43:00 Client-0-ccc6572c-3f39-4df8-9175-106a0b8d44a7 INFO     [14,   250] loss: 0.039
10-10-2022 16:43:10 Client-0-ccc6572c-3f39-4df8-9175-106a0b8d44a7 INFO     [14,   300] loss: 0.041
10-10-2022 16:43:20 Client-0-ccc6572c-3f39-4df8-9175-106a0b8d44a7 INFO     [14,   350] loss: 0.031
10-10-2022 16:43:29 Client-0-ccc6572c-3f39-4df8-9175-106a0b8d44a7 INFO     [14,   400] loss: 0.043
10-10-2022 16:43:39 Client-0-ccc6572c-3f39-4df8-9175-106a0b8d44a7 INFO     [14,   450] loss: 0.047
10-10-2022 16:43:49 Client-0-ccc6572c-3f39-4df8-9175-106a0b8d44a7 INFO     [14,   500] loss: 0.037
10-10-2022 16:43:59 Client-0-ccc6572c-3f39-4df8-9175-106a0b8d44a7 INFO     [14,   550] loss: 0.038
10-10-2022 16:44:18 Client-0-ccc6572c-3f39-4df8-9175-106a0b8d44a7 INFO     [15,     0] loss: 0.001
10-10-2022 16:44:28 Client-0-ccc6572c-3f39-4df8-9175-106a0b8d44a7 INFO     [15,    50] loss: 0.029
10-10-2022

10-10-2022 16:57:31 Client-0-ccc6572c-3f39-4df8-9175-106a0b8d44a7 INFO     [21,   150] loss: 0.029
10-10-2022 16:57:41 Client-0-ccc6572c-3f39-4df8-9175-106a0b8d44a7 INFO     [21,   200] loss: 0.027
10-10-2022 16:57:50 Client-0-ccc6572c-3f39-4df8-9175-106a0b8d44a7 INFO     [21,   250] loss: 0.037
10-10-2022 16:58:00 Client-0-ccc6572c-3f39-4df8-9175-106a0b8d44a7 INFO     [21,   300] loss: 0.036
10-10-2022 16:58:10 Client-0-ccc6572c-3f39-4df8-9175-106a0b8d44a7 INFO     [21,   350] loss: 0.029
10-10-2022 16:58:20 Client-0-ccc6572c-3f39-4df8-9175-106a0b8d44a7 INFO     [21,   400] loss: 0.034
10-10-2022 16:58:29 Client-0-ccc6572c-3f39-4df8-9175-106a0b8d44a7 INFO     [21,   450] loss: 0.020
10-10-2022 16:58:39 Client-0-ccc6572c-3f39-4df8-9175-106a0b8d44a7 INFO     [21,   500] loss: 0.031
10-10-2022 16:58:49 Client-0-ccc6572c-3f39-4df8-9175-106a0b8d44a7 INFO     [21,   550] loss: 0.024
10-10-2022 16:59:09 Client-0-ccc6572c-3f39-4df8-9175-106a0b8d44a7 INFO     [22,     0] loss: 0.000
10-10-2022

       0.96067416, 0.98914224, 0.98341969, 0.96058091, 0.95436508]), class_recall=array([0.98469388, 0.99030837, 0.95251938, 0.97029703, 0.97148676,
       0.95852018, 0.95093946, 0.92315175, 0.95071869, 0.95341923]), confusion_mat=array([[ 965,    0,    1,    0,    2,    3,    1,    1,    2,    5],
       [   0, 1124,    2,    5,    0,    0,    2,    0,    2,    0],
       [   5,   14,  983,   11,    8,    3,    1,    1,    6,    0],
       [   1,    0,    5,  980,    0,   13,    0,    3,    4,    4],
       [   0,    4,    0,    0,  954,    0,    3,    1,    0,   20],
       [   2,    2,    6,   19,    0,  855,    1,    2,    3,    2],
       [  20,    3,    3,    1,    6,    3,  911,    1,    8,    2],
       [   1,   21,   28,   10,    3,    3,    0,  949,    3,   10],
       [   7,    1,    6,   12,    7,    8,    2,    2,  926,    3],
       [   3,    3,    3,    5,   16,    2,    0,    5,   10,  962]])), EpochData(epoch_id=4, num_epochs=26, duration_train=500637, duration_test=9

       0.98542601, 0.94467641, 0.9688716 , 0.97946612, 0.98909812]), confusion_mat=array([[ 972,    1,    0,    0,    1,    1,    1,    1,    2,    1],
       [   0, 1126,    2,    1,    0,    2,    0,    1,    3,    0],
       [   4,    2, 1019,    0,    0,    0,    0,    2,    4,    1],
       [   0,    0,   13,  964,    0,   25,    0,    1,    2,    5],
       [   0,    0,    1,    0,  954,    1,    2,    0,    2,   22],
       [   0,    0,    2,    6,    0,  879,    0,    1,    2,    2],
       [   2,    5,    3,    0,    3,   34,  905,    0,    5,    1],
       [   3,    1,   13,    6,    1,    0,    0,  996,    2,    6],
       [   2,    0,    4,    1,    2,    5,    1,    1,  954,    4],
       [   2,    0,    0,    0,    4,    1,    0,    2,    2,  998]])), EpochData(epoch_id=11, num_epochs=26, duration_train=1392944, duration_test=9685, loss_train=0.04309675117023289, accuracy=97.71, loss=0.8079815423116088, class_precision=array([0.99575372, 0.98603839, 0.95962441, 0.98692153

       [   0, 1123,    4,    4,    1,    0,    2,    1,    0,    0],
       [   5,    1, 1015,    3,    0,    0,    2,    6,    0,    0],
       [   0,    0,    3, 1001,    0,    1,    0,    1,    3,    1],
       [   0,    0,    0,    0,  961,    0,    5,    2,    1,   13],
       [   2,    0,    2,   25,    0,  854,    4,    1,    4,    0],
       [   2,    4,    2,    0,    5,    5,  937,    0,    2,    1],
       [   0,    1,    8,    6,    0,    0,    0, 1013,    0,    0],
       [   3,    0,    6,    2,    2,    3,    1,    2,  952,    3],
       [   2,    0,    0,   10,    5,    4,    0,   10,    2,  976]])), EpochData(epoch_id=18, num_epochs=26, duration_train=2283441, duration_test=9624, loss_train=0.03720758270006627, accuracy=97.96, loss=0.7529150564223528, class_precision=array([0.98385469, 0.98859649, 0.97413793, 0.97722772, 0.9957492 ,
       0.96179039, 0.98407643, 0.97870281, 0.98153846, 0.96924603]), class_recall=array([0.99489796, 0.99295154, 0.98546512, 0.97722772, 0

       [   3,    5,  988,   12,    1,    2,    5,   13,    2,    1],
       [   0,    0,    2,  992,    0,   11,    0,    3,    2,    0],
       [   0,    0,    0,    0,  960,    0,    4,    0,    1,   17],
       [   2,    0,    1,    9,    0,  877,    1,    0,    2,    0],
       [   2,    2,    1,    1,    4,    6,  940,    0,    2,    0],
       [   2,    9,    1,    4,    0,    0,    0, 1009,    1,    2],
       [   5,    0,    2,    5,    1,    4,    1,    2,  947,    7],
       [   2,    0,    0,    2,    6,    3,    1,    3,    2,  990]])), EpochData(epoch_id=25, num_epochs=26, duration_train=3173739, duration_test=9697, loss_train=0.02141757219331339, accuracy=98.42, loss=0.6478457497432828, class_precision=array([0.98785425, 0.98863636, 0.99119374, 0.98898899, 0.97114428,
       0.98964327, 0.98844538, 0.96499527, 0.98654244, 0.98597194]), class_recall=array([0.99591837, 0.99647577, 0.98158915, 0.97821782, 0.99389002,
       0.96412556, 0.9822547 , 0.9922179 , 0.97843943, 0.9

## Copy experiment results from the extractor

Extractor holds the experiment results in the format that can be processedby TensorBoard.
In order to download it to the local machine, execute:

In [21]:
EXTRACTOR_POD_NAME=$(kubectl get pods -n test -l "app.kubernetes.io/name=fltk.extractor" -o jsonpath="{.items[0].metadata.name}")

kubectl cp -n test $EXTRACTOR_POD_NAME:/opt/federation-lab/logging ./logging

tar: Removing leading `/' from member names


# Cleanup

## Removing orchestrator

In [22]:
helm uninstall -n test experiment-orchestrator

W1010 19:10:18.272893   14620 gcp.go:120] WARNING: the gcp auth plugin is deprecated in v1.22+, unavailable in v1.25+; use gcloud instead.
To learn more, consult https://cloud.google.com/blog/products/containers-kubernetes/kubectl-auth-changes-in-gke
release "experiment-orchestrator" uninstalled


## Removing extractor

IMPORTANT: Removing extractor chart will result in deleting the already collected experiment results, stored in the NFS!

In [23]:
helm uninstall extractor -n test

W1010 19:10:22.215474   14634 gcp.go:120] WARNING: the gcp auth plugin is deprecated in v1.22+, unavailable in v1.25+; use gcloud instead.
To learn more, consult https://cloud.google.com/blog/products/containers-kubernetes/kubectl-auth-changes-in-gke
release "extractor" uninstalled


## Wrapping up

To scale down the cluster nodepools, run the cell below. This will scale the node pools down and remove all the experiments deployed (on the cluster).

1. Experiments cannot be restarted.
2. Experiment logs will not persist deletion.


In [24]:
# This will remove all information and logs as well.
kubectl delete pytorchjobs.kubeflow.org --all-namespaces --all

pytorchjob.kubeflow.org "trainjob-2a03676e-093c-49be-9154-db2da09e8eb8" deleted
pytorchjob.kubeflow.org "trainjob-ccc6572c-3f39-4df8-9175-106a0b8d44a7" deleted


In [25]:
gcloud container clusters resize $CLUSTER_NAME --node-pool $DEFAULT_POOL \
    --num-nodes 0 --region $REGION --quiet

gcloud container clusters resize $CLUSTER_NAME --node-pool $EXPERIMENT_POOL \
    --num-nodes 0 --region $REGION --quiet

Resizing fltk-testbed-cluster...done.


Updated [https://container.googleapis.com/v1/projects/test-bed-fltk-jerrit/zones/us-central1-c/clusters/fltk-testbed-cluster].
Resizing fltk-testbed-cluster...done.                                          
Updated [https://container.googleapis.com/v1/projects/test-bed-fltk-jerrit/zones/us-central1-c/clusters/fltk-testbed-cluster].
